In [36]:
import heapq
import numpy as np
from icecream import ic
from random import choice
from tqdm.auto import tqdm
from collections import namedtuple

In [37]:
PUZZLE_DIM = 3
Action = namedtuple('Action', ['pos1', 'pos2'])

In [38]:
def available_actions(state: np.ndarray) -> list['Action']:
    x_start, y_start = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x_start > 0:
        actions.append(Action((x_start, y_start), (x_start-1, y_start)))
    if x_start < PUZZLE_DIM - 1:
        actions.append(Action((x_start, y_start), (x_start+1, y_start)))
    if y_start > 0:
        actions.append(Action((x_start, y_start), (x_start, y_start-1)))
    if y_start < PUZZLE_DIM - 1:
        actions.append(Action((x_start, y_start), (x_start, y_start+1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


In [39]:
def manhattan_distance(state: np.ndarray) -> int:
    total_distance = 0
    # row: (n-1) // PUZZLE_DIM, column: n - PUZZLE_DIM * row - 1
    correct_pos = {n : ((n-1) // PUZZLE_DIM, n - PUZZLE_DIM * ((n-1) // PUZZLE_DIM) - 1) for n in range(1, PUZZLE_DIM**2)}
    correct_pos[0] = (PUZZLE_DIM-1, PUZZLE_DIM-1)
    for n in range(PUZZLE_DIM**2):
        x_current, y_current = [int(_[0]) for _ in np.where(state == n)]
        total_distance += abs(x_current - correct_pos[n][0]) + abs(y_current - correct_pos[n][1])
    return total_distance

In [40]:
RANDOMIZE_STEPS = 1000
GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
print(GOAL_STATE)
initial_state = GOAL_STATE.copy()
for step in range(RANDOMIZE_STEPS): # use tqdm
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))
    
h_distance = manhattan_distance(initial_state)

[[1 2 3]
 [4 5 6]
 [7 8 0]]


In [41]:
class Node:
    def __init__(self, state: np.ndarray, parent, g: int, h: int) -> None:
        self.state = state
        self.parent = parent
        self.g = g
        self.h = h

    def __lt__(self, next):
        return self.g + self.h < next.g + next.h

In [42]:
frontier = []
explored = set()
heapq.heappush(frontier, Node(initial_state, None, 0, h_distance))

while frontier:
    current_node = heapq.heappop(frontier)
    if np.array_equal(current_node.state, GOAL_STATE):
        path = []
        node = current_node
        while node.parent != None:
            path.append(node.state)
            node = node.parent
        path.append(initial_state)
        print(path[::-1])
        break
    for action in available_actions(current_node.state):
        new_state = do_action(current_node.state, action)
        if tuple(new_state.flatten()) not in explored:
            explored.add(tuple(new_state.flatten()))
            heuristic_distance = manhattan_distance(new_state)
            heapq.heappush(frontier, Node(new_state, current_node, current_node.g + 1, heuristic_distance))

print(f"\nSolve {PUZZLE_DIM*PUZZLE_DIM-1}-puzzle in {len(path)} moves.")


[array([[8, 6, 4],
       [1, 7, 3],
       [5, 2, 0]]), array([[8, 6, 4],
       [1, 7, 3],
       [5, 0, 2]]), array([[8, 6, 4],
       [1, 0, 3],
       [5, 7, 2]]), array([[8, 0, 4],
       [1, 6, 3],
       [5, 7, 2]]), array([[8, 4, 0],
       [1, 6, 3],
       [5, 7, 2]]), array([[8, 4, 3],
       [1, 6, 0],
       [5, 7, 2]]), array([[8, 4, 3],
       [1, 0, 6],
       [5, 7, 2]]), array([[8, 4, 3],
       [0, 1, 6],
       [5, 7, 2]]), array([[0, 4, 3],
       [8, 1, 6],
       [5, 7, 2]]), array([[4, 0, 3],
       [8, 1, 6],
       [5, 7, 2]]), array([[4, 1, 3],
       [8, 0, 6],
       [5, 7, 2]]), array([[4, 1, 3],
       [8, 7, 6],
       [5, 0, 2]]), array([[4, 1, 3],
       [8, 7, 6],
       [0, 5, 2]]), array([[4, 1, 3],
       [0, 7, 6],
       [8, 5, 2]]), array([[4, 1, 3],
       [7, 0, 6],
       [8, 5, 2]]), array([[4, 1, 3],
       [7, 5, 6],
       [8, 0, 2]]), array([[4, 1, 3],
       [7, 5, 6],
       [8, 2, 0]]), array([[4, 1, 3],
       [7, 5, 0],
       [8, 